# Training exploration

In [5]:
from glob import glob
import pickle

In [8]:
files = sorted(glob("../training/*.pkl"), reverse=True)
with open(files[0], 'rb') as f:
    state = pickle.load(f)

In [23]:
for i, gen in enumerate(state.training_gens):
    print(f"### Training gen {gen.child_gen} from {gen.source_gen}. Tournament results:")
    print("```")
    print(gen.tournament.scores_table())
    print("```")
    print("")

### Training gen 1 from 0. Tournament results:
```
gen1: 46.0
uniform: 41.0
random: 33.0
```

### Training gen 2 from 1. Tournament results:
```
gen2: 107.5
gen1: 45.5
uniform: 45.0
random: 42.0
```

### Training gen 3 from 2. Tournament results:
```
gen2: 129.0
gen3: 113.0
gen1: 70.5
random: 52.5
uniform: 35.0
```

### Training gen 4 from 2. Tournament results:
```
gen4: 135.0
gen2: 94.0
gen3: 92.5
gen1: 48.0
random: 30.5
```

### Training gen 5 from 4. Tournament results:
```
gen2: 100.0
gen4: 100.0
gen3: 100.0
gen5: 70.0
gen1: 30.0
```

### Training gen 6 from 2. Tournament results:
```
gen3: 100.0
gen6: 100.0
gen4: 80.0
gen2: 80.0
gen5: 40.0
```

### Training gen 7 from 3. Tournament results:
```
gen4: 120.0
gen6: 100.0
gen7: 80.0
gen3: 60.0
gen2: 40.0
```

### Training gen 8 from 4. Tournament results:
```
gen4: 120.0
gen8: 90.0
gen6: 80.0
gen7: 70.0
gen3: 40.0
```

### Training gen 9 from 4. Tournament results:
```
gen4: 120.0
gen8: 90.0
gen9: 80.0
gen6: 60.0
gen7: 50.0
```

### 

In [25]:
# Total training positions
tot = sum(len(gen.training_data) for gen in state.training_gens)
print(f"Total training positions: {tot}")

Total training positions: 2361492
